In [5]:
%%timeit
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
10.8 s ± 565 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [2]:
%%timeit
import keras_ocr
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5
11.4 s

In [1]:
import keras_ocr
import json
import os
import pickle

In [2]:
class SimpleBox():
    def __init__(self, stupid_box):
        self.word = stupid_box[0]
        self.top = min([i[1] for i in stupid_box[1]])
        self.left = min([i[0] for i in stupid_box[1]])
        self.bottom = max([i[1] for i in stupid_box[1]])
        self.right = max([i[0] for i in stupid_box[1]])
        self.height = self.bottom - self.top

In [3]:
class BigBox():
    def __init__(self, list_of_simple_boxes):
        self.original = list_of_simple_boxes
        self.top = min([i.top for i in list_of_simple_boxes])
        self.left = min([i.left for i in list_of_simple_boxes])
        self.bottom = max([i.bottom for i in list_of_simple_boxes])
        self.right = max([i.right for i in list_of_simple_boxes])
        self.height = self.bottom - self.top
        self.center = ((self.right - self.left)/2) + self.left

In [4]:
def create_text_groups(text_from_image, bottom_of_row_gap=10, next_word_distance=10, next_row_distance=10, margin_closeness=10):
    texts = [SimpleBox(text) for text in text_from_image]
    # could sort texts by finding min of right bottom point (x + y) or, like, pythagorean theorem
    
    row_groups = []
    while texts:
        # grab first word
        text = texts[0]
        
        # collect all text on same "line"
        temp_row = [j for j in texts if abs(text.bottom - j.bottom) < text.height/2]
        
       # sort the list in place in order of left to right
        temp_row.sort(key=lambda x: x.left, reverse=False)
#         print([i.word for i in temp_row])
        
        # Find large breaks within words
        # Initialize new list with first word
        temp_row_2 = [temp_row[0]]
        # Loop through rest of words in temp_row
        for index, word in enumerate(temp_row[1:]):
            # Is word close to previous?
            if abs(word.left - temp_row[index].right) < text.height/2:
                temp_row_2.append(word)
            else:
                break
        
#         print('final temp_row_2', [word.word for word in temp_row_2])
        # add word group to collection
        row_groups.append(temp_row_2)
        # remove those words from possible words
        [texts.pop(texts.index(word)) for word in temp_row_2]
    
    # Now to check if rows are near each other... somehow...
    rows = [BigBox(row) for row in row_groups]
    
    text_groups = []
    while rows:
        matching_row = rows[0]
        
        group = [matching_row]
        for new_row in rows[1:]:
            # is top of new_row near bottom of our row?
            if (new_row.top - matching_row.bottom) < (matching_row.height*.75):
                # and is center of row within the margins of our row?
                if  matching_row.left < new_row.center < matching_row.right:
                    group.append(new_row)
                    matching_row = new_row
                    
        text_groups.append(group)
        # remove those rows from possible rows
        [rows.pop(rows.index(row)) for row in group]
    
    return text_groups
    

In [5]:
def remove_positional_information(text_groups):
    """
    Takes the result from create_text_groups and returns only the text.
    TODO: build this object in the create_text_groups function to prevent
    this double loop.
    """
    text = {}
    text['groupings'] = [[[word.word for word in line.original] for line in group] for group in text_groups]
    text['raw_text'] = '\n\n'.join(['\n'.join([' '.join([word.word for word in line.original]) for line in group]) for group in text_groups])
    return text

In [6]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\dale\.keras-ocr\craft_mlt_25k.h5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Looking for C:\Users\dale\.keras-ocr\crnn_kurapan.h5


In [7]:
media_dir = '/Users/dale/Downloads/r7PpXQD4/'
filenames = os.listdir(media_dir)

In [8]:
images = []
image_names = []
errors = []
for file in filenames:
    try:
        images.append(keras_ocr.tools.read(media_dir + file))
        image_names.append(file)
    except Exception as e:
        print(e, end='', flush=True)
        errors.append((file, e))

In [9]:
errors

[]

In [10]:
# ALL images
to_do = [(image_name, image) for image_name, image in zip(image_names, images)]

In [32]:
# Just remaining images
#...
with open('remaining.pkl', 'rb') as remaining_file:
    test = pickle.load(remaining_file)

In [33]:
print(len(test))
to_do = test

18


In [34]:
num = 0
total = len(to_do)

with open('reddit_meme_predictions.ndjson', 'a') as results_file:

    while to_do:
        image_name, image = to_do.pop(0)
        num += 1
        
        try:
            prediction = pipeline.recognize([image])
            text_groups = remove_positional_information(create_text_groups(prediction[0]))
            results_file.write(json.dumps({
                                           'name': image_name,
                                           'text' : text_groups,
                                          }))
            results_file.write('\n')
            print(str(num) + ' of ' + str(total) + ' completed', end='\r', flush=True)

        except Exception as e:
            print(e, flush=True) 
            results_file.write(json.dumps({
                                           'name': image_name,
                                           'error' : str(e),
                                          }))
            results_file.write('\n')
        
        # update remaining (cause this thing takes forever!)
        with open('remaining.pkl', 'wb') as remaining_file:
            pickle.dump(to_do, remaining_file)

In [38]:
data = []
with open('reddit_meme_predictions.ndjson', 'r') as results_file:
    for line in results_file.readlines():
        data.append(json.loads(line))

In [39]:
for i in data:
    if 'text' in i.keys():
        print(i['name'], ':')
        print(i['text']['raw_text'])
        print('_________________________________')

0rlo319o4w671.jpg :
nobody

reddit live streamers
_________________________________
2scwgdir4w671.jpg :
me after eating 2 bags of popcorni 2 filled to the brim bowls of cereal and 3 cadburys oreo chocolate bars wondering why im not losing weight and have diarrhea

ting ne think
_________________________________
58gqpnjk3w671.gif :
oi mate we goin to
tesco ye wan anything
_________________________________
6s1o3fn84w671.png :
4y0 me after i planted a cashew
into sand and its already been 134 seconds

let the dark harvest beginuu
_________________________________
7aax1qe84w671.jpg :
cheering for the
villain in disney
movies

cheering for
the villain in
starwars

cheering for
the villain in
1943 disney
shows

made with mematic
_________________________________
9u7nfnmg4w671.jpg :
igbtq
the moon

80957 likes

igbtq its official the moon is gay

of apansystbattie by ashamelesspinup makeup byn
view all 20927 comments
september 24 2020

more

me and
the boys
_________________________________
9

In [40]:
len(data)

25